In [2]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
import nltk
from xml.dom import minidom
from xml.etree import cElementTree as ElementTree
import os



# Gensim Corpus and Tf.idf Model

# Gensim Corpus and Tf.Idf Model

## Document Reader

In [4]:

def documentReader(path, queries = False):
    """
    DocString
    :return: Nothing
    """
    documents_path = os.path.join(os.getcwd(), path)
    documentos = {}
    for filename in os.listdir(documents_path):
        file_path = os.path.join(documents_path, filename)
        xmldoc = minidom.parse(file_path)
        id = xmldoc.getElementsByTagName('public')[0].attributes['publicId'].value
        title = '' if queries else xmldoc.getElementsByTagName('fileDesc')[0].attributes['title'].value
        data = next(ElementTree.parse(file_path).iter('raw')).text
        documentos[id] = (title + ' ' + data).replace(u'\xa0', u' ').replace('\n', ' ')

    return documentos
documentos = documentReader('docs/docs-raw-texts')
NRO_DOCS = len(documentos)
DOCS_IDs = list(documentos.keys())
print(list(documentos.items())[0])

('d001', 'William Beaumont and the Human Digestion William Beaumont and the Human Digestion.  William Beaumont: Physiology of digestion Image Source.  On November 21, 1785, US-American surgeon William Beaumont was born. He became best known as “Father of Gastric Physiology” following his research on human digestion. William Beaumont was born in Lebanon, Connecticut and became a physician. He served as a surgeon’s mate in the Army during the War of 1812. He opened a private practice in Plattsburgh, New York, but rejoined the Army as a surgeon in 1819. Beaumont was stationed at Fort Mackinac on Mackinac Island in Michigan in the early 1820s when it existed to protect the interests of the American Fur Company. The fort became the refuge for a wounded 19-year-old French-Canadian fur trader named Alexis St. Martin when a shotgun went off by accident in the American Fur Company store at close range June 6th, 1822. St. Martin’s wound was quite serious because his stomach was perforated and se

### Tokenize

In [5]:
p = PorterStemmer()
def process(text):
    doc_nor = text.lower()
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = p.stem_sentence(doc_sw)
    return nltk.word_tokenize(doc_stem)

docDict = []
for key, doc in documentos.items():
    docDict.append(process(doc))

docDict[0][:5]

NameError: name 'documentos' is not defined

In [6]:
dictionary = corpora.Dictionary(docDict)
dictionary.save('docs/midict.dict')
print(dictionary.token2id['information'])

76


In [7]:
##  Market Matrix format
# Step 1: Build the corpus from big file
class MyCorpus():
    def __init__(self, documents):
        self.documents = documents
    def __iter__(self):
        for key, doc in self.documents.items():
            yield dictionary.doc2bow(process(doc))

corpus_memory_friendly = MyCorpus(documentos)
corpora.MmCorpus.serialize("docs/corpus.mm",corpus_memory_friendly)

#### Read Maket Matrix format from disk

In [8]:
corpus = corpora.MmCorpus("docs/corpus.mm")
# No hacer esto en una implementacion real
for doc in corpus:
    print(doc[:10])
    break

[(0, 20.0), (1, 21.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0)]


#### Build tf.idf model from corpus

In [3]:
dictionary = corpora.Dictionary.load('docs/midict.dict')
corpus = corpora.MmCorpus('docs/corpus.mm')
tfidf = models.TfidfModel(corpus)

In [6]:
#Test to verify correct reading
query = "Machine learning"
query_doc_bow = dictionary.doc2bow(process(query)) # Important: Same corpus preprocess
print(query_doc_bow)
print(tfidf[query_doc_bow])

[(241, 1), (5809, 1)]
[(241, 0.2642196547502339), (5809, 0.9644625311766483)]


#### Make similarity matrix

In [7]:
index = similarities.MatrixSimilarity(tfidf[corpus])
index.save('docs/similmatrix.index')
print('Finished')

Finished


## Querying and validating

In [ ]:
index = similarities.MatrixSimilarity.load('docs/similmatrix.index')
sims = index[tfidf[query_doc_bow]]
print(list(enumerate(sims))[:10])


### Read and proccess queries